<a href="https://colab.research.google.com/github/shhesterka04/Quantum-Insights/blob/polina/maximum_subarray_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The maximum subarray problem**

# Немного о самой проблеме
Задан массив чисел, цель состоит в том, чтобы найти непрерывный подмассив с наибольшей суммой. Эта проблема имеет приложения в обработке сигналов и финансовом анализе.

# Что гпт посоветует для решения проблемы?  

Алгоритм квантовой приближенной оптимизации (QAOA) может быть наиболее подходящим для решения этой проблемы.

Было показано, что QAOA эффективен при решении различных задач оптимизации и известен своей способностью обеспечивать решения, близкие к оптимальным, для комбинаторных задач. Кроме того, было продемонстрировано, что QAOA имеет лучшую производительность, чем два других алгоритма, с точки зрения времени выполнения и масштабируемости, которые являются важными факторами для решения проблемы максимального подмассива.



# Алгоритм QAOA для решения задачи о максимальном подмассиве с использованием формы Изинга гамильтониана :

1. **Определите количество кубитов, необходимых для представления задачи.**

2. **Закодируйте задачу бинарной оптимизации как гамильтониан Изинга.**

 Модель Изинга — это математическая модель ферромагнетизма в статистической механике, описывающая взаимодействия между спинами на решетке. Гамильтониан для модели Изинга определяется выражением:

$H_{Ising} = - \sum_{i,j} J_{i,j} \sigma_i \sigma_j - \sum_{i} h_i \sigma_i$

где $\sigma_i$ — спин $i$-й частицы, $J_{i,j}$ — сила взаимодействия между частицами $i$ и $j$, $h_i$ — внешнее поле, действующее на $i$-я частица.

В случае задачи о максимальном подмассиве мы можем сопоставить бинарные переменные $x_i$ со спинами, установив $x_i = \frac{1}{2}(1 - \sigma_i)$, где $\sigma_i \in \{- 1,1\}$. Тогда мы можем выразить гамильтониан задачи о максимальном подмассиве в форме Изинга как:

$H_{max-subarray} = - \sum_{i,j} w_{i,j} \frac{1}{4} (1 - \sigma_i) (1 - \sigma_j) - \frac{1}{2 } \sum_{i} w_{i,i} \sigma_i$

где $w_{i,j}$ — вес ребра между вершинами $i$ и $j$, а $\sigma_i$ — спин, соответствующий $i$-й переменной.

3. **Создайте квантовую схему, подготавливающую начальное состояние для алгоритма QAOA. Это может быть суперпозиция всех возможных состояний кубитов.**

4. **Определить QAOA-оператор $U(\gamma, \beta)$, состоящий из двух частей:** гамильтониана стоимости и гамильтониана смесителя. Гамильтониан стоимости — это гамильтониан Изинга для задачи максимального подмассива, а гамильтониан смесителя — это простой оператор, который используется для изменения состояния кубитов. Общий гамильтониан — это оператор $X$, примененный к каждому кубиту. Оператор QAOA определяется как:

$U(\gamma, \beta) = e^{-i\beta P}e^{-i\gamma H_{max-subarray}}$

где $\gamma = (\gamma_1, \gamma_2, \dots, \gamma_p)$ и $\beta = (\beta_1, \beta_2, \dots, \beta_p)$ — оптимизируемые параметры, $H_{max- subarray}$ — гамильтониан Изинга для задачи о максимальном подмассиве, а $P$ — смешивающий гамильтониан, который определяется как:

$P = \sum\limits_{i=1}^{n} X_i$

где $X_i$ — оператор Паули-Х, действующий на кубит $i$.

5. **Применить оператор QAOA к начальному состоянию, используя квантовую схему, созданную на шаге 3.** Количество применений оператора определяется глубиной алгоритма QAOA, которая обозначается $p$.

6. **Измерьте кубиты и запишите результаты измерения.**

7. **С помощью классического оптимизатора найти оптимальные значения параметров $\gamma$ и $\beta$, минимизирующие целевую функцию.** Целевой функцией является среднее значение стоимостного гамильтониана $H_{max-subarray}$ относительно конечного состояния, полученного в результате применения оператора QAOA.

8. **Используйте оптимальные параметры для расчета максимального подмассива.**

In [ ]:
import numpy as np
from qiskit import Aer, execute
from qiskit.aqua import aqua_globals
from qiskit.aqua.operators import PauliSumOp
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua.algorithms import QAOA


def find_maximum_subarray(array):
    # Define the Ising Hamiltonian for the maximum subarray problem
    n = len(array)
    pauli_list = []
    for i in range(n):
        for j in range(i, n):
            if i == j:
                wp = -0.5 * array[i]
            else:
                wp = -array[i] * array[j]
            pauli_list.append([f'Z{i}Z{j}', wp])
    operator = PauliSumOp.from_list(pauli_list)
    
    # Define the initial state for the QAOA algorithm
    init_state = np.ones(2 ** n) / np.sqrt(2 ** n)
    
    # Define the variational form for the QAOA algorithm
    p = 1  # depth of the QAOA algorithm
    variational_form = RYRZ(n, depth=p)
    
    # Define the QAOA algorithm
    optimizer = COBYLA()
    algorithm = QAOA(operator=operator, optimizer=optimizer, p=p, initial_point=[0]*2*p,
                     mixer=variational_form, initial_state=init_state)
    
    # Execute the QAOA algorithm and get the optimal parameters
    backend = Aer.get_backend('qasm_simulator')
    shots = 1000
    aqua_globals.random_seed = 10598
    result = algorithm.run(backend, shots=shots)
    optimal_params = result.optimal_params
    
    # Calculate the maximum subarray using the optimal parameters
    statevector = algorithm.construct_circuit(optimal_params).statevector()
    max_subarray = np.argmax(np.abs(statevector))
    
    return max_subarray
